<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/kubrac_relevant_parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import os
import math
import csv

import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_datasets as tfds
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import tqdm
from ipywidgets import IntProgress



Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
class Arguments(object):
    dataset = # possible data sets to try: 'mnist', 'svhn_cropped', 'cifar10', 'cifar100',
    lossChoice = # 'all_triplet', 'hard_triplet', 'semihard_triplet', 'contrastive'
    save_path = # enter a path to save
    excel_path = ''
    runBreg = # True      
    trial_index = # enter an id for the experiment
    
    def add_args(self, args_dict):
        [setattr(self, key, args_dict[key]) for key in args_dict]
        
    def get_args(self):
        arg_list = {var: getattr(self, var) for var in dir(self)
                    if not callable(getattr(self, var))
                    and not var.startswith("__")}
        return arg_list
    def print_args(self):
        arg_dict = self.get_args()
        print_str = ''
        for arg in arg_dict:
            print_str += arg + ' = ' + str(arg_dict[arg]) + '\n'
        return print_str
    
    @classmethod
    def init_csv(self, name):
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        path = os.path.join(self.save_path, self.dataset + '_' + name + '.csv')
        print(path)
        headers = ['model_number', 'lossChoice',  
                   'test_acc_breg', 'test_auc_breg',
                  'test_avp_breg', 'test_acc_euc',
                  'test_auc_euc', 'test_avp_euc'] + list(self.get_args(self).keys()) 
        with open(path, 'w') as csv_file:
            csv_writer = csv.writer(csv_file, delimiter=',')
            csv_writer.writerow(headers)
        self.excel_path = path
        return path

    def init_excel(self, name):
        if not os.path.isfile(self.save_path + name):
            wb = Workbook()
            sheet1 = wb.add_sheet('Sheet 1')
            for i, h in enumerate(self.headers):
                sheet1.write(i, 0, self.headers)
            wb.save(self.save_path + name)
            self.excel_path = self.save_path + name
            return self.excel_path
            
    def addrow_to_excel(self, val_list):
        #with open(excel_path) as f:
        #    reader = csv.reader(f)
        data = [self.trial_index, self.lossChoice] + val_list + list(self.get_args().values())  #, '%.3f'%(knn_train), 
        with open(self.excel_path, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(data)

arguments = Arguments()
excel_path = Arguments.init_csv('results')
print(excel_path)

In [4]:
def _pairwise_divergences(embed):

    max_out = tf.math.argmax(embed, 1, output_type=tf.dtypes.int32)
    one_to_n = tf.range(tf.shape(embed)[0], dtype=tf.dtypes.int32)
    max_indices = tf.transpose(tf.stack([one_to_n, max_out]))
    max_values = tf.gather_nd(embed, max_indices)
    max_values_repeated = tf.transpose(tf.reshape(tf.tile(max_values, [tf.shape(embed)[0]]), [tf.shape(embed)[0], tf.shape(embed)[0]]))
    repeated_max_out = tf.tile(max_out, [tf.shape(embed)[0]])
    repeated_one_to_n = tf.tile(one_to_n, [tf.shape(embed)[0]])
    mat_rotn = tf.reshape(tf.transpose(tf.reshape(repeated_one_to_n, [tf.shape(embed)[0], tf.shape(embed)[0]])), [-1])
    new_max_indices = tf.transpose(tf.stack([mat_rotn, repeated_max_out]))
    new_max_values = tf.gather_nd(embed, new_max_indices)
    reshaped_new_max_values = tf.reshape(new_max_values, [tf.shape(embed)[0], tf.shape(embed)[0]])
    div_matrix = tf.maximum(tf.subtract(max_values_repeated, reshaped_new_max_values), 0.0)  
    return div_matrix

In [5]:
_pairwise_divergences

<function __main__._pairwise_divergences(embed)>

In [6]:
def _get_anchor_positive_triplet_mask(labels):

    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    mask = tf.logical_and(indices_not_equal, labels_equal)
    return mask

In [7]:
def _get_anchor_negative_triplet_mask(labels):

    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    mask = tf.logical_not(labels_equal)
    return mask

def _get_triplet_mask(labels):

    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)
    distinct_indices = tf.logical_and(tf.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)
    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))
    mask = tf.logical_and(distinct_indices, valid_labels)
    return mask

In [17]:
def batch_all_triplet_loss(labels, embeddings, margin, squared=False, breg=False):

    pairwise_dist = _pairwise_divergences(embeddings)
    anchor_positive_dist = tf.expand_dims(pairwise_dist, 2)
    assert anchor_positive_dist.shape[2] == 1, "{}".format(anchor_positive_dist.shape)
    anchor_negative_dist = tf.expand_dims(pairwise_dist, 1)
    assert anchor_negative_dist.shape[1] == 1, "{}".format(anchor_negative_dist.shape)
    
    triplet_loss = anchor_positive_dist - anchor_negative_dist + margin
    mask = _get_triplet_mask(labels)
    mask = tf.to_float(mask)
    triplet_loss = tf.multiply(mask, triplet_loss)

    triplet_loss = tf.maximum(triplet_loss, 0.0)
    valid_triplets = tf.to_float(tf.greater(triplet_loss, 1e-16))
    num_positive_triplets = tf.reduce_sum(valid_triplets)
    num_valid_triplets = tf.reduce_sum(mask)
    fraction_positive_triplets = num_positive_triplets / (num_valid_triplets + 1e-16)
    triplet_loss = tf.reduce_sum(triplet_loss) / (num_positive_triplets + 1e-16)

    return triplet_loss, fraction_positive_triplets

# only batch all triplet loss

In [18]:
def get_loss(runBreg, margin, pretrain, y, k, out):
        loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, k, dtype=tf.float32), logits=out))
        loss2, fpt = batch_all_triplet_loss(y, out, margin, squared=True, breg=runBreg)
        loss = pretrain*loss1 + (1-pretrain)*loss2
        return loss1, loss2, loss